In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
DIR_TRAIN = '../Data/Use/Train/'
os.listdir(DIR_TRAIN)
print('# Species in train set:',len(os.listdir(DIR_TRAIN)))

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_datagen = train_datagen.flow_from_directory(DIR_TRAIN,
                                                target_size=(224,224), 
                                                color_mode='rgb', 
                                                class_mode='categorical', 
                                                shuffle=False)

In [ ]:
X = np.concatenate([train_datagen.next()[0] for i in range(train_datagen.__len__())])

In [ ]:
CASPIAN_TERN = X[0:100]
D_ARNAUDS_BARBET = X[100:200]
DARK_EYED_JUNCO = X[200:300]
HOUSE_FINCH = X[300:400]
OVENBIRD = X[400:500]
OYSTER_CATCHER = X[500:600]
RED_TAILED_HAWK = X[600:700]
SWINHOES_PHEASENT = X[700:800]
VIOLET_GREEN_SWALLOW = X[800:900]
WOOD_DUCK = X[900:1000]

#### K-Fold Autoencoder training (Spoiler: It doesn't work)

In [ ]:
def k_fold_training(list_of_data):

    lis = []
    lis2 = []

    for x in list_of_data:
        # Save the n-1 images to the list
        for i in range(len(x)-1):
            lis.append(x[i+1])
            lis2.append(x[i])
    
    tup = (lis, lis2)

    return tup

tup = k_fold_training([WOOD_DUCK, CASPIAN_TERN, D_ARNAUDS_BARBET, DARK_EYED_JUNCO, HOUSE_FINCH, OVENBIRD, OYSTER_CATCHER, RED_TAILED_HAWK, SWINHOES_PHEASENT, VIOLET_GREEN_SWALLOW])

print(np.shape(tup))

In [ ]:
plt.imshow(tup[0][350])

In [ ]:
plt.imshow(tup[1][350])

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(tup)

type(dataset)

In [ ]:
dataset = dataset.map(lambda x, y: (tf.cast(x, tf.float32), tf.cast(y, tf.float32)))
dataset = dataset.map(lambda x, y: (tf.expand_dims(x, 0), tf.expand_dims(y, 0)))

dataset.element_spec

In [ ]:
expe = dataset.shuffle(buffer_size=10)

In [ ]:
plt.subplot(1,2,1)
plt.imshow(tf.squeeze(expe.as_numpy_iterator().next()[0],[0]))
plt.subplot(1,2,2)
plt.imshow(tf.squeeze(expe.as_numpy_iterator().next()[1],[0]))

In [ ]:
# Prepare both datasets 

normal_dataset = dataset.prefetch(20)

dataset_with_shuffle = expe.prefetch(20)

_______________

In [ ]:
class autoencoder(tf.keras.Model):
    def __init__(self, latent_dim):
        super(autoencoder, self).__init__()
        
        self.encoder = [
            tf.keras.layers.InputLayer(input_shape=(None,224,224,3)),
            tf.keras.layers.Conv2D(filters=4,kernel_size=3,strides=2,padding='valid', activation='relu'),
            tf.keras.layers.Conv2D(filters=8,kernel_size=3,strides=2,padding='valid', activation='relu'),
            tf.keras.layers.Conv2D(filters=16,kernel_size=3,strides=2,padding='valid', activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(latent_dim*latent_dim*3)
        ]

        self.decoder = [
            tf.keras.layers.Reshape(target_shape=(latent_dim,latent_dim,3)),
            tf.keras.layers.Conv2DTranspose(filters=16,kernel_size=2,strides=2,padding='valid', activation='relu'),
            tf.keras.layers.Conv2DTranspose(filters=8,kernel_size=2,strides=2,padding='valid', activation='relu'),
            tf.keras.layers.Conv2DTranspose(filters=4,kernel_size=2,strides=2,padding='valid', activation='relu'),
            tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=2, strides=2, padding='valid', activation='sigmoid'),
            tf.keras.layers.Reshape(target_shape=(224,224,3))
        ]

    def encode(self, x):
        for layer in self.encoder:
            x = layer(x)
        return x

    def decode(self, x):
        for layer in self.decoder:
            x = layer(x)
        return x

    @tf.function
    def call(self, input):
        x = self.encode(input)
        x = self.decode(x)
        return x

In [ ]:
auto = autoencoder(latent_dim=14)

auto.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
auto.build(input_shape=(None,224,224,3))

In [ ]:
auto.fit(dataset, epochs=10)

In [ ]:
bird = DARK_EYED_JUNCO

plt.figure(figsize=(20,5))
for i in range(10):
    plt.subplot(1,10,i+1)
    image_to_predict = tf.expand_dims(bird[i], 0).numpy()
    embeddings = auto.encode(image_to_predict).numpy()  
    plt.imshow(embeddings.reshape(14,14,3))

#### Normal Autoencode (Same Inputs as Targets) training

In [ ]:
auto2 = autoencoder(latent_dim=14)

auto2.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
auto2.build(input_shape=(None,224,224,3))

In [ ]:
auto2.fit(X, X, epochs=10)